In [52]:
from google.colab import drive
import cv2
import numpy as np
import gradio as gr
from scipy.signal import medfilt
from matplotlib import pyplot as plt

# Mount Google Drive
drive.mount('/content/drive', force_remount=True)

def refine_skyline(mask):
    """
    Refines the skyline in the given binary mask by adjusting column-wise transitions.
    """
    num_rows, num_cols = mask.shape

    for col_index in range(num_cols):
        current_column = mask[:, col_index]
        smoothed_column = medfilt(current_column, 19)

        try:
            first_zero_index = np.where(smoothed_column == 0)[0][0]
            first_one_index = np.where(smoothed_column == 1)[0][0]

            # Adjust skyline if there is a significant transition
            if first_zero_index > 20:
                mask[first_one_index:first_zero_index, col_index] = 1
                mask[first_zero_index:, col_index] = 0
                mask[:first_one_index, col_index] = 0

        except IndexError:
            continue

    return mask

def get_sky_region_gradient(img):
    """
    Extracts the sky region using gradient-based methods.
    """
    height, width, _ = img.shape

    # Convert the image to grayscale
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Blur the image to reduce noise
    img_blurred = cv2.blur(img_gray, (9, 3))

    # Apply median blur to further reduce noise
    cv2.medianBlur(img_blurred, 5)

    # Apply Laplacian filter to enhance edges
    laplacian_output = cv2.Laplacian(img_blurred, cv2.CV_8U)

    # Create a gradient mask based on Laplacian filter output
    gradient_mask = (laplacian_output < 6).astype(np.uint8)

    # Define a morphological kernel
    morph_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (9, 3))

    # Erode the gradient mask to remove small artifacts
    gradient_mask_eroded = cv2.morphologyEx(gradient_mask, cv2.MORPH_ERODE, morph_kernel)

    # Copy the original mask for further processing
    mask_copy = gradient_mask_eroded.copy()

    # Refine the mask using the refine_skyline function
    refined_mask = refine_skyline(mask_copy)

    # Apply the refined mask to the original image
    after_img = cv2.bitwise_and(img, img, mask=refined_mask)

    return after_img, refined_mask

def display_binary_mask(ax, masked_image):
    """
    Displays the binary mask with white for sky and black for non-sky.
    """
    binary_mask = (masked_image > 0).astype(np.uint8) * 255
    ax.imshow(binary_mask, cmap="gray", vmin=0, vmax=255)
    ax.set_title("Final Result: Binary Mask")

def get_sky_region(image_path):
    """
    Extracts and visualizes the sky region from the given image.
    """
    # Read the image
    image = cv2.imread(image_path)

    if image is not None:
        # Create a 2x2 grid for subplots
        fig, axs = plt.subplots(1, 2, figsize=(12, 6))

        # Display the original image
        axs[0].imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        axs[0].set_title("Original Image")

        # Skyline detection and morphological operations
        masked_image, binary_mask = get_sky_region_gradient(image)

        # Display the binary mask as the final result
        display_binary_mask(axs[1], binary_mask)

        # Display the final result
        plt.tight_layout()
        plt.show()
    else:
        print("Error: Unable to read the image.")

def gr_sky_region_extraction(image):
    # Convert Gradio Image type to numpy array
    image_array = np.array(image)

    # Sky region extraction
    img_gray = cv2.cvtColor(image_array, cv2.COLOR_BGR2GRAY)
    masked_image, binary_mask = get_sky_region_gradient(image_array)

    # Convert the output image to RGB for Gradio compatibility
    output_image_rgb = cv2.cvtColor(masked_image, cv2.COLOR_BGR2RGB)

    return output_image_rgb

# Define the Gradio interface
demo = gr.Interface(
    fn=gr_sky_region_extraction,
    inputs="image",
    outputs="image",
    title="Sky Pixel Identification",
    description="Result： Light part indicates sky pixels, black for non-sky",
    allow_flagging="never",
    live=True,
    )

# Launch the Gradio interface
demo.launch()


Mounted at /content/drive

Thanks for being a Gradio user! If you have questions or feedback, please join our Discord server and chat with us: https://discord.gg/feTf9x3ZSB
Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://31f77ba41c568a0b62.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
